In [5]:
import sys
!{sys.executable} -m pip install gym


     ---------------------------------------- 0.0/721.7 kB ? eta -:--:--
     ------------------------------------  716.8/721.7 kB 15.0 MB/s eta 0:00:01
     ------------------------------------- 721.7/721.7 kB 15.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for cloudpickle>=1.2.0 from https://files.pythonhosted.org/packages/96/43/dae06432d0c4b1dc9e9149ad37b4ca8384cf6eb7700cd9215b177b914f0a/cloudpickle-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for gym-notices>=0.0.4 from https://files.pythonhosted.org/packages/25/26/d786c6bec30fe6110fd3d22c9a273a2a0e56c0b73b93e25ea1af5a53243b/gym_notices-0.0.8-py3-none-any.whl.metadata
  Created


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import gym
import numpy as np
from gym import spaces

class ThermostatEnvironment(gym.Env):
    def __init__(self):
        self.action.space = spaces.Discrete(3) #increase, decrease, maintain
        self.observation_space = spaces.Box(low=18, high=np.array([25]), dtype=np.float32) #min temp = 18, max temp = 25
        self.set_point = 18
        self.temperature = 18
        self.max_temp = 25
        self.min_temp = 18
        self.reward = 0
        self.done = False
    
    def step(self, action):
        if action == 0:
            self.temperature += 1 #increase temperature
            self.reward = 2 #reward for increasing temperature
        elif action == 1:
            self.temperature -= 1 #decrease temperature
            self.reward = -1  #penalty for decreasing temperature
        else:
            pass
        if self.temperature < self.min_temp:
            self.temperature = self.min_temp
        elif self.temperature > self.max_temp:
            self.temperature = self.max_temp
        if self.temperature == self.set_point:
            self.reward = 1
        else:
            self.reward = 0
        return self.temperature, self.reward, self.done, {}

    def reset(self):
        self.temperature = 18
        return self.temperature
    

In [8]:
import tensorflow as tf

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class DQN:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self.build_model()

    # neural network model 

    #return action based on state
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    # remember experience
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = np.random.choice(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [10]:
# train model

    def build_model(self):